# Auswertung der Fragebogen

In [1]:
import json
import pathlib
import pandas as pd

In [2]:
path = pathlib.Path("beispiel-daten")

In [3]:
def survey_data(directory):
    for proband in directory.glob("*"):
        pseudonym = proband.name
        data = list()
        for file in pathlib.Path(proband, "fragebogen-daten").glob("*.json"):
            with file.open("r", encoding="utf-8") as survey:
                answers = pd.Series(json.load(survey))
                # Save name:
                answers.name = file.stem
                if "nasa" in answers.name or "quesi" in answers.name:
                    # Drop the website row:
                    answers = answers.drop("website")
                    # Need integers:
                    answers = answers.apply(int)
                data.append(answers)
        yield {"pseudonym": pseudonym, "data": data}

In [4]:
def aggregate(data, website):
    tlx = list()
    quesi = list()
    pre = list()
    post = list()
    for proband in data:
        pseudonym = proband["pseudonym"]
        data = proband["data"]
        for survey in data:
            if survey.name == f"{website}-nasa":
                score = survey * 5
                score = score.sum() / 6
                tlx.append(score)
            elif survey.name == f"{website}-quesi":
                # TODO
                pass
            elif survey.name == "pre":
                survey.name = pseudonym
                pre.append(survey)
            elif survey.name == "post":
                survey.name = pseudonym
                post.append(survey)
    return pd.Series(tlx), quesi, pd.DataFrame(pre), pd.DataFrame(post)

In [5]:
data = survey_data(path)
tlx, quesi, pre, post = aggregate(data, "shirtinator")

In [6]:
pre

,Geschlecht,Alter,bildungsabschluss,Derzeitige Beschäftigung bzw. Studienfach,Websites schon einmal benutzt
kesi26,männlich,23,Universitäts-/Hochschulabschluss,Digital Humanities,nein
mano15,männlich,23,Universitäts-/Hochschulabschluss,Digital Humanities,nein


In [7]:
quesi

[]

In [8]:
tlx

0    60.833333
1    60.833333
dtype: float64

In [10]:
tlx.median()

60.833333333333336

In [9]:
post

,Bessere Website,Positiv bei shirtinator.de,Negativ bei shirtinator.de,Positiv bei spreadshirt.de,Negativ bei spreadshirt.de
kesi26,beide gleich gut,Der Editor,,Schönes Design,Der Editor
mano15,beide gleich gut,Der Editor,,Schönes Design,Der Editor
